In [ ]:
#!pip install dash
#!pip install plotly
#!pip install dash_mantine_components
#!pip install dash_bootstrap_components
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from sklearn import datasets
from pandas import DataFrame,concat
import plotly.graph_objects as go

iris = datasets.load_iris()
mapspecies = {
    0:'Iris-setosa',
    1:'Iris-versicolor',
    2:"Iris-virginica"
  }


dft = DataFrame(iris.target, columns=['Species'])
dft.Species = dft.Species.map(mapspecies)
dft.reset_index(drop=True, inplace=True)
dfd = DataFrame(iris.data, columns=['SepalLength','SepalWidth','PetalLength','PetalWidth'])
dfd.reset_index(drop=True, inplace=True)
df = concat([dft,dfd], axis=1)

colors = {
    'background_maindiv': '#F3FF33',
    'background_plot': '#B4BD26'
}

# Create a Dash web application
app = dash.Dash(__name__)
# Define the layout of the app
app.layout = html.Div(style={'backgroundColor': colors['background_maindiv']}, children=[
    html.H1(children='My Dashboard on IRIS Species', style={'textAlign': 'center', 'font-family':'Helve',"fontSize": "48px"}),
    html.Div([
      dcc.Dropdown(['SepalWidth', 'SepalLength', 'PetalLength', 'PetalWidth'], 'SepalWidth', id='x-axis-variable'),
      dcc.Dropdown(['SepalWidth', 'SepalLength', 'PetalLength', 'PetalWidth'], 'SepalLength', id='y-axis-variable'),
      dcc.RadioItems(['Iris-setosa', 'Iris-versicolor','Iris-virginica'], value='Iris-virginica', id="selected-species", style={'textAlign': 'left', 'fontSize': 23})
    ], style={'width': '20%', 'display': 'inline-block',  'verticalAlign': 'middle'}),
    html.Div([
    dcc.Graph(figure={}, id='our-first-graph')
    ], style={'width': '40%', 'display': 'inline-block',  'verticalAlign': 'middle'}),
    html.Div([
      dcc.Graph(figure={}, id='our-second-graph')
    ], style={'width': '40%', 'display': 'inline-block',  'verticalAlign': 'middle'}),
])
@app.callback(
    Output(component_id='our-first-graph', component_property='figure'),
    Output(component_id='our-second-graph', component_property='figure'),
    [
        Input(component_id="selected-species", component_property='value'),
        Input(component_id="x-axis-variable", component_property='value'),
        Input(component_id="y-axis-variable", component_property='value')
    ]
)
def update_output(selected_species_value,x_axis_variable,y_axis_variable):
  # First chart
  ourfirstplot = go.Figure()
  sdf = df[df["Species"]==selected_species_value].copy()
  ourfirstplot.add_trace(go.Scatter(x=sdf[x_axis_variable],y=sdf[y_axis_variable],text=sdf['Species'],mode='markers'))
  ourfirstplot.update_layout(xaxis={'title': x_axis_variable}, yaxis={'title': y_axis_variable}, title='Iris Sepal dimensions', paper_bgcolor=colors['background_plot'])

  # Second Chart
  sdf2 = df.groupby('Species').size()
  oursecondplot = go.Figure([go.Bar(x=sdf2.index, y=sdf2.values)])

  return ourfirstplot,oursecondplot
if __name__ == '__main__':
    app.run_server(mode='inline')  # Run the app in the Jupyter Notebook inline mode